In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

### Make scene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, None,
                INDY_IP)]
              , connection_list=[False])
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"panda0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


## Preapare background

In [3]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.3,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
grip0 = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
gscene.show_pose(crob.home_pose)

Please create a subscriber to the marker


## Door

In [4]:
from pkg.utils.code_scraps import *

In [5]:
door, door_s = add_door(pscene, center=(0.7,-0.2,0.5), color=(0.8,0.4,0.2,1))
knob = gscene.create_safe(GEOTYPE.BOX, 'knob', link_name="base_link",
                          center=(-door.dims[0]/2-0.01,-door.dims[1]/2+0.05,0), rpy=(0,-np.pi/2,0), dims=(0.02,0.02,0.02),
                          fixed=False, collision=True, color=(0.8,0.8,0.8,1), parent="door")

knob_s = add_lever(pscene, knob, dims=(0.02,0.1,0.02), knob_offset=0.04)

### prepare planner

In [6]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [7]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan.motion_filters = [GraspChecker(pscene), ReachChecker(pscene)]
mplan.incremental_constraint_motion = True

### Plan

In [8]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

['door', 'knob', 'lever_grip']
(0, 0, 'knob')


In [9]:
goal_nodes = [(2, 2,'grip0')]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 18/36 agents
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
try: 0 - (0, 0, 'knob')->(0, 0, 'grip0')
try: 0 - (0, 0, 'knob')->(0, 0, 'grip0')
try transition motion
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
try transition motion
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
try: 0 - (0, 0, 'knob')->(1, 0, 'knob')
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
try: 0 - (0, 0, 'knob')->(0, 0, 'grip0')
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
try transition motion
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
result: 0 - (0, 0, 'knob')->(1, 0, 'knob') = fail
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
try: 0 - (0, 0, 'knob')->(0, 0, 'grip0')
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
try transition motion
transition motion tried: True
try: 0 - (0, 0, 'knob')->(1, 0, 'knob')
result: 0 - (0, 0, 'knob')->(0,

try: 15 - (0, 1, 'grip0')->(0, 2, 'grip0')
result: 11 - (0, 1, 'knob')->(1, 1, 'knob') = fail
try: 10 - (0, 1, 'grip0')->(1, 1, 'grip0')
try: 3 - (0, 1, 'grip0')->(0, 2, 'grip0')
constrained motion tried: False
result: 5 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
try: 6 - (0, 0, 'grip0')->(0, 1, 'grip0')
Motion Plan Failure
try: 5 - (0, 1, 'grip0')->(1, 1, 'grip0')
try: 6 - (0, 0, 'grip0')->(1, 0, 'grip0')
result: 5 - (0, 1, 'grip0')->(0, 2, 'grip0') = fail
result: 6 - (0, 0, 'grip0')->(0, 1, 'grip0') = success
try: 16 - (0, 1, 'grip0')->(0, 2, 'grip0')
branching: 6->16 (0.62/30.0 s, steps/err: 1(8.7468624115 ms)/0)
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
result: 10 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
result: 5 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
result: 10 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
try: 15 - (0, 1, 'grip0')->(1, 1, 'grip0')
joint max
joint max
try: 3 - (0, 1, 'grip0')->(1, 1, 'grip0')
constrained motion tried: False
try: 0 - (0, 0, 'knob')->(1, 0, 'knob')
Mo

try: 12 - (0, 1, 'knob')->(1, 1, 'knob')
try: 23 - (0, 0, 'grip0')->(1, 0, 'grip0')
constrained motion tried: False
try: 14 - (0, 1, 'knob')->(1, 1, 'knob')
Motion Plan Failure
result: 11 - (0, 1, 'knob')->(1, 1, 'knob') = fail
try: 22 - (0, 1, 'grip0')->(0, 2, 'grip0')
try: 15 - (0, 1, 'grip0')->(1, 1, 'grip0')
Not available:knob-0
result: 16 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
result: 2 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
result: 20 - (0, 1, 'grip0')->(0, 2, 'grip0') = fail
result: 23 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
np_exclude:['r1']
result: 12 - (0, 1, 'knob')->(1, 1, 'knob') = fail
result: 14 - (0, 1, 'knob')->(1, 1, 'knob') = fail
bd_exclude:knob_plug
Not available:knob-0
np_exclude:['r1']
try: 16 - (0, 1, 'grip0')->(1, 1, 'grip0')
try: 5 - (0, 1, 'grip0')->(0, 2, 'grip0')
try: 12 - (0, 1, 'knob')->(0, 2, 'knob')
try: 4 - (0, 0, 'grip0')->(0, 1, 'grip0')
bd_exclude:knob_plug
result: 15 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
try: 5 - (0, 1, 'grip0')->(0, 1,

result: 17 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
bd_exclude:door_hinge
try: 2 - (0, 0, 'grip0')->(1, 0, 'grip0')
============== Non-available transition: sample again =====================
try: 30 - (0, 2, 'grip0')->(0, 2, 'knob')
============== Non-available transition: sample again =====================
link transfer between same links: ['lever_grip'] - panda0
try: 31 - (1, 2, 'grip0')->(2, 2, 'grip0')
result: 2 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
Motion Plan Failure
Not available:door-0
transition motion tried: True
try: 29 - (0, 0, 'knob')->(1, 0, 'knob')
result: 30 - (0, 2, 'grip0')->(0, 2, 'knob') = fail
np_exclude:['h0']
try: 29 - (0, 0, 'knob')->(0, 0, 'grip0')
bd_exclude:door_hinge
transition motion tried: True
result: 28 - (0, 0, 'grip0')->(0, 0, 'knob') = success
branching: 28->33 (1.29/30.0 s, steps/err: 2(84.1701030731 ms)/0.00201871650126)
end
try transition motion
result: 6 - (0, 0, 'grip0')->(0, 0, 'knob') = success
try: 7 - (0, 1, 'knob')->(0, 2, 'knob')
====

branching: 3->47 (1.54/30.0 s, steps/err: 80(919.847011566 ms)/1.56140077057e-06)
try: 47 - (0, 2, 'grip0')->(1, 2, 'grip0')
Motion Plan Failure
branching: 44->48 (1.54/30.0 s, steps/err: 1(6.56890869141 ms)/0)
try: 1 - (0, 0, 'grip0')->(1, 0, 'grip0')
result: 31 - (1, 2, 'grip0')->(1, 2, 'knob') = fail
branching: 16->46 (1.54/30.0 s, steps/err: 17(77.8691768646 ms)/0.00115047590612)
result: 47 - (0, 2, 'grip0')->(1, 2, 'grip0') = success
try: 14 - (0, 1, 'knob')->(1, 1, 'knob')
try: 49 - (1, 2, 'grip0')->(2, 2, 'grip0')
branching: 47->49 (1.57/30.0 s, steps/err: 1(6.04104995728 ms)/0)
result: 45 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
result: 1 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
try: 32 - (0, 0, 'knob')->(1, 0, 'knob')
result: 14 - (0, 1, 'knob')->(1, 1, 'knob') = fail
try: 29 - (0, 0, 'knob')->(1, 0, 'knob')
try: 19 - (0, 0, 'knob')->(0, 1, 'knob')
try: 33 - (0, 0, 'knob')->(1, 0, 'knob')
try: 25 - (0, 1, 'knob')->(0, 1, 'grip0')
joint max
Not available:knob-0
try transition

result: 28 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
result: 2 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
try: 60 - (0, 0, 'grip0')->(0, 1, 'grip0')
result: 29 - (0, 0, 'knob')->(1, 0, 'knob') = fail
try: 0 - (0, 0, 'knob')->(1, 0, 'knob')
try: 35 - (0, 2, 'grip0')->(1, 2, 'grip0')
result: 45 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
Not available:door-0
result: 8 - (0, 0, 'knob')->(1, 0, 'knob') = fail
try: 31 - (1, 2, 'grip0')->(2, 2, 'grip0')
try: 36 - (0, 0, 'grip0')->(0, 0, 'knob')
np_exclude:['h0']
try: 56 - (0, 2, 'grip0')->(1, 2, 'grip0')
joint max
constrained motion tried: False
Not available:knob-0
result: 60 - (0, 0, 'grip0')->(0, 1, 'grip0') = success
np_exclude:['r1']
branching: 60->62 (1.87/30.0 s, steps/err: 1(7.28583335876 ms)/0)
Motion Plan Failure
result: 35 - (0, 2, 'grip0')->(1, 2, 'grip0') = success
x
bd_exclude:door_hinge
result: 43 - (1, 2, 'grip0')->(2, 2, 'grip0') = fail
branching: 35->61 (1.89/30.0 s, steps/err: 1(6.66499137878 ms)/0)
try transition motion
try:

try: 2 - (0, 0, 'grip0')->(0, 0, 'knob')
branching: 7->72 (2.14/30.0 s, steps/err: 3(65.0148391724 ms)/0.00125042606954)
try: 71 - (1, 2, 'grip0')->(2, 2, 'grip0')
result: 2 - (0, 0, 'grip0')->(0, 0, 'knob') = success
try: 72 - (0, 1, 'grip0')->(1, 1, 'grip0')
transition motion tried: True
branching: 2->73 (2.16/30.0 s, steps/err: 1(5.99217414856 ms)/0)
result: 52 - (0, 0, 'grip0')->(0, 1, 'grip0') = success
branching: 52->74 (2.16/30.0 s, steps/err: 15(55.5980205536 ms)/0.00176349499999)
try: 74 - (0, 1, 'grip0')->(0, 2, 'grip0')
result: 72 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
try: 14 - (0, 1, 'knob')->(0, 2, 'knob')
Not available:door-0
np_exclude:['h0']
bd_exclude:door_hinge
result: 14 - (0, 1, 'knob')->(0, 2, 'knob') = fail
try: 47 - (0, 2, 'grip0')->(1, 2, 'grip0')
joint max
============== Non-available transition: sample again =====================
constrained motion tried: False
result: 47 - (0, 2, 'grip0')->(1, 2, 'grip0') = success
try: 13 - (0, 1, 'knob')->(0, 2, 'knob')

++ adding return motion to acquired answer ++
result: 75 - (1, 2, 'grip0')->(2, 2, 'grip0') = fail
try: 4 - (0, 0, 'grip0')->(0, 1, 'grip0')
Not available:knob-0
np_exclude:['r1']
result: 4 - (0, 0, 'grip0')->(0, 1, 'grip0') = success
bd_exclude:knob_plug
branching: 4->82 (2.58/30.0 s, steps/err: 1(7.10415840149 ms)/0)
end
constrained motion tried: True
try: 82 - (0, 1, 'grip0')->(1, 1, 'grip0')
x
transition motion tried: False
Goal reached
============== Non-available transition: sample again =====================
Motion Plan Failure
result: 66 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
try: 19 - (0, 0, 'knob')->(0, 0, 'grip0')
result: 44 - (0, 0, 'grip0')->(0, 0, 'knob') = fail
joint max
branching: 66->83 (2.6/30.0 s, steps/err: 81(618.597984314 ms)/0.000178981241815)
constrained motion tried: False
result: 82 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
try transition motion
Motion Plan Failure
transition motion tried: True
++ adding return motion to acquired answer ++
x
try: 58 - (1

try: 96 - (1, 2, 'grip0')->(2, 2, 'grip0')
branching: 95->96 (2.86/30.0 s, steps/err: 1(5.93996047974 ms)/0)
result: 11 - (0, 1, 'knob')->(1, 1, 'knob') = fail
======================= terminated 9: required answers acquired  (2.9/30.0) ===============================
end
======================= terminated 14: Stop called from other agent  (2.9/30.0) ===============================
======================= terminated 3: Stop called from other agent  (2.9/30.0) ===============================
Not available:knob-0
constrained motion tried: True
np_exclude:['r1']
bd_exclude:knob_plug
======================= terminated 8: Stop called from other agent  (2.9/30.0) ===============================
============== Non-available transition: sample again =====================
result: 74 - (0, 1, 'grip0')->(0, 2, 'grip0') = success
branching: 74->98 (2.89/30.0 s, steps/err: 80(705.028057098 ms)/8.09463577963e-06)
try: 8 - (0, 0, 'knob')->(1, 0, 'knob')
Goal reached
======================= terminated 

++ adding return motion to acquired answer ++
end
constrained motion tried: True
x
======================= terminated 13: required answers acquired  (3.3/30.0) ===============================
Goal reached
result: 93 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 93->110 (3.3/30.0 s, steps/err: 81(530.738115311 ms)/0.000178981241815)
++ adding return motion to acquired answer ++
Goal reached
Goal reached
======================= terminated 0: required answers acquired  (3.4/30.0) ===============================
Goal reached
======================= terminated 16: required answers acquired  (3.5/30.0) ===============================
======================= terminated 11: required answers acquired  (3.5/30.0) ===============================
[ERROR] Non-joined subprocesses: [4]
========================== FINISHED (3.6 / 30.0 s) ==============================]


In [11]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

In [12]:
snode_schedule = tplan.get_best_schedule(at_home=False)
ppline.play_schedule(snode_schedule)

(0, 0, 'knob')->(0, 0, 'grip0')
(0, 0, 'grip0')->(0, 1, 'grip0')
(0, 1, 'grip0')->(0, 1, 'knob')
(0, 1, 'knob')->(0, 1, 'grip0')
(0, 1, 'grip0')->(0, 2, 'grip0')
(0, 2, 'grip0')->(1, 2, 'grip0')
(1, 2, 'grip0')->(2, 2, 'grip0')


In [56]:
door_hinge.check_available(list2dict(from_state.Q, gscene.joint_names))

False

In [57]:
door_hinge.geometry.is_controlled()

False

In [59]:
door_hinge.geometry.link_name

'base_link'

### push back

In [14]:
gscene.show_pose(crob.home_pose)
term_state = snode_schedule[-1].state
print(pscene.subject_name_list)
print(term_state.node)

['door', 'knob', 'lever_grip']
(2, 2, 'grip0')


In [15]:
goal_nodes = [(1, 1,'knob')]
ppline.search(term_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 18/36 agents
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
try: 0 - (2, 2, 'grip0')->(2, 2, 'knob')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
try: 0 - (2, 2, 'grip0')->(2, 2, 'knob')
result: 0 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
link transfer between same links: ['lever_grip'] - panda0
link transfer between same links: ['lever_grip'] - panda0
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
Motion Plan Failure
Motion Plan Failure
result: 0 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
result: 0 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'knob')
link transfer between same links: ['lever_grip'] - panda0
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
Motion Plan Failure
result: 0 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer

result: 0 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
Motion Plan Failure
link transfer between same links: ['lever_grip'] - panda0
link transfer between same links: ['lever_grip'] - panda0
end
Motion Plan Failure
branching: 0->5 (0.94/30.0 s, steps/err: 81(792.647123337 ms)/0.000258457597)
constrained motion tried: True
Motion Plan Failure
result: 0 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
result: 5 - (1, 2, 'grip0')->(1, 2, 'knob') = fail
result: 4 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
branching: 0->6 (0.96/30.0 s, steps/err: 81(714.611053467 ms)/0.000258457597)
result: 3 - (1, 2, 'grip0')->(1, 2, 'knob') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
result: 1 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
link transfer between same links: ['knob'] - panda0
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
Motion Plan Failure
try: 6 - (1, 2, 'grip0')->(1, 2, 'grip0')
resul

try: 13 - (1, 2, 'grip0')->(1, 2, 'knob')
link transfer between same links: ['lever_grip'] - panda0
try: 14 - (1, 2, 'grip0')->(1, 2, 'grip0')
result: 11 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
try: 6 - (1, 2, 'grip0')->(1, 2, 'grip0')
np_exclude:['h0']
link transfer between same links: ['knob'] - panda0
bd_exclude:door_hinge
Motion Plan Failure
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
result: 13 - (1, 2, 'grip0')->(1, 2, 'knob') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
Motion Plan Failure
end
constrained motion tried: True
try: 0 - (2, 2, 'grip0')->(2, 2, 'knob')
link transfer between same links: ['knob'] - panda0
result: 14 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
Motion Plan Failure
============== Non-available transition: sample again =====================
result: 0 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
result: 0 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
result: 6 - (1, 2, 'grip0')->(1, 1, 'g

end
end
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
constrained motion tried: True
constrained motion tried: True
result: 3 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
result: 0 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
x
branching: 0->20 (1.84/30.0 s, steps/err: 81(828.655004501 ms)/0.000258457597)
result: 3 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
try: 20 - (1, 2, 'grip0')->(1, 2, 'grip0')
branching: 3->21 (1.84/30.0 s, steps/err: 81(843.335151672 ms)/0.000259761880768)
============== Non-available transition: sample again =====================
end
try: 21 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
end
constrained motion tried: True
Motion Plan Failure
result: 0 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
constrained motion tried: True
Not available:door-0
result: 20 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
end
np_exclude:['h0']
try: 22 - (1, 2, 'grip0')->(1, 2, 'grip0')
x
bd_exclude:door_hinge
Not availab

Motion Plan Failure
============== Non-available transition: sample again =====================
result: 11 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
try: 17 - (1, 2, 'grip0')->(2, 2, 'grip0')
try: 26 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
result: 26 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
Not available:door-0
np_exclude:['h0']
bd_exclude:door_hinge
============== Non-available transition: sample again =====================
try: 21 - (2, 2, 'grip0')->(2, 2, 'grip0')
end
constrained motion tried: True
result: 0 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
branching: 0->27 (2.16/30.0 s, steps/err: 81(625.092029572 ms)/0.000258457597)
try: 27 - (1, 2, 'grip0')->(1, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
result: 27 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
try: 6 - (1, 2, 'grip0')->(1, 2, 'knob')
try: 26 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['lev

...

try: 154 - (2, 2, 'grip0')->(2, 2, 'knob')
try: 159 - (1, 2, 'grip0')->(1, 2, 'grip0')
end
try: 115 - (1, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['lever_grip'] - panda0
Not available:door-0
link transfer between same links: ['knob'] - panda0
constrained motion tried: True
np_exclude:['h0']
Motion Plan Failure
Motion Plan Failure
result: 159 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
bd_exclude:door_hinge
try: 160 - (1, 2, 'grip0')->(1, 2, 'grip0')
result: 97 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
result: 154 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
branching: 97->160 (7.69/30.0 s, steps/err: 81(775.587081909 ms)/0.000281153729718)
try: 5 - (1, 2, 'grip0')->(1, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
============== Non-available transition: sample again =====================
Motion Plan Failure
end
try: 120 - (1, 2, 'grip0')->(2, 2, 'grip0')
try: 19 - (1, 2, 'grip0')->(2, 2, 'grip0')
constrained motion tried: True
link transfer betwee

Motion Plan Failure
result: 75 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
Not available:door-0
np_exclude:['h0']
bd_exclude:door_hinge
end
constrained motion tried: True
============== Non-available transition: sample again =====================
x
try: 79 - (1, 2, 'grip0')->(2, 2, 'grip0')
result: 74 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 74->166 (8.02/30.0 s, steps/err: 81(623.203039169 ms)/0.000255489359844)
try: 43 - (1, 2, 'grip0')->(2, 2, 'grip0')
end
constrained motion tried: True
x
result: 73 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 73->167 (8.03/30.0 s, steps/err: 81(659.972906113 ms)/0.000255489359844)
try: 24 - (2, 2, 'grip0')->(2, 2, 'knob')
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
result: 24 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
Not available:door-0
np_exclude:['h0']
bd_exclude:door_hinge
============== Non-available transition: sample again =====================
try: 41 - (2, 2, 'grip0')->(2, 2, 'knob')


try: 73 - (1, 2, 'grip0')->(2, 2, 'grip0')
x
result: 19 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 19->179 (8.5/30.0 s, steps/err: 81(791.02396965 ms)/0.000259761880768)
try: 172 - (2, 2, 'grip0')->(2, 2, 'knob')
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
result: 172 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'knob')
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
result: 0 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
try: 143 - (1, 2, 'grip0')->(2, 2, 'grip0')
end
constrained motion tried: True
x
result: 10 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 10->180 (8.57/30.0 s, steps/err: 81(778.527021408 ms)/0.000259761880768)
try: 158 - (1, 2, 'grip0')->(2, 2, 'grip0')
end
constrained motion tried: True
x
result: 79 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 79->181 (8.59/30.0 s, steps/err: 81(574.740886688 ms)/0.000255489359844)
try: 59 - (1, 2, 'grip0')->(1, 

end
constrained motion tried: True
x
result: 94 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 94->190 (8.88/30.0 s, steps/err: 81(723.870992661 ms)/0.000255489359844)
end
constrained motion tried: True
x
result: 155 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 155->191 (8.94/30.0 s, steps/err: 81(700.966119766 ms)/0.000255489359844)
end
end
constrained motion tried: True
constrained motion tried: True
x
x
result: 73 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 73->192 (8.96/30.0 s, steps/err: 81(466.472148895 ms)/0.000255489359844)
result: 128 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 128->193 (8.96/30.0 s, steps/err: 81(520.180940628 ms)/0.000255489359844)
end
constrained motion tried: True
x
result: 66 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 66->194 (8.99/30.0 s, steps/err: 81(715.376853943 ms)/0.000255489359844)
end
constrained motion tried: True
x
result: 143 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 143->1

======================= terminated 13: node queue empty 14th  (9.7/30.0) ===============================
======================= terminated 15: node queue empty 15th  (9.7/30.0) ===============================
======================= terminated 7: node queue empty 16th  (9.8/30.0) ===============================
======================= terminated 5: node queue empty 17th  (9.8/30.0) ===============================
generate table - Geometry
generate table - Handle
generate table - Binder
generate table - Object
generate table - PlanConditions
generate table - PlanList
button clicked
(2, 2, 'grip0')->(1, 2, 'grip0')
button action done
button clicked
(2, 2, 'grip0')->(1, 2, 'grip0')
button action done
========================== FINISHED (30.3 / 30.0 s) ==============================]
button clicked
(2, 2, 'grip0')->(1, 2, 'grip0')
button action done
button clicked
(2, 2, 'grip0')->(1, 2, 'grip0')
(1, 2, 'grip0')->(2, 2, 'grip0')
(2, 2, 'grip0')->(1, 2, 'grip0')
(1, 2, 'grip0')->(2, 2, 'gr

In [20]:
snode_schedule = tplan.get_best_schedule()
ppline.play_schedule(snode_schedule)

(2, 'knob')->(2, 'grip0')
(2, 'grip0')->(1, 'grip0')
(1, 'grip0')->(1, 'knob')
(1, 'knob')->(1, 'knob')
